In [ ]:
# Overview of geospatial data, coordinate systems and geometry types.  

In [53]:
from sedona.spark import *
from pyspark.sql.functions import col, when, expr
from sedona.sql.st_functions import ST_IsValid, ST_IsValidReason, ST_MakeValid
from pyspark.sql import DataFrame

In [3]:
config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
# Read and create basic data frames  - Pranav

sedona.read

In [ ]:
# Vector drivers  - Pranav

# GeoJSON
# CSV
# Shapefile

# Show others in comments

In [ ]:
# Raster - Pranav

# Raster

# Show others

In [ ]:
# Intro to cloud native formats - Pranav

# Demo the speed on this 
# COG from LANDSAT - STAC

In [ ]:
# Intro to Iceberg - Pranav

In [ ]:
# Transforming data with non-native readers - in slides

# Wherobots Fundamentals - Constructing

WherobotsDB provides a powerful set of functions to construct geometries. You can either create them from scratch using raw coordinate values (literals) or by parsing standard geospatial data formats like WKT and WKB.

---

### Creating from Coordinates

These functions build geometries directly from numerical inputs.

* `ST_MakePoint(x, y, [z], [m])`: Creates a **Point** geometry from its x and y coordinates. You can also optionally provide a z-coordinate (for elevation) and an m-coordinate (a measure value).

* `ST_MakeEnvelope(xmin, ymin, xmax, ymax)`: Creates a rectangular **Polygon** that represents a bounding box, or "envelope," from the coordinates of two opposing corners.

* `ST_LineStringFromText(text, delimiter)`: This function builds a LineString from a flat string of comma-separated coordinates, like `'x1, y1, x2, y2, ...'`. This provides a fast way to create line geometries directly from raw text data without needing the formal structure of WKT.

* `ST_PolygonFromText(text, delimiter)`: Similarly, the ST_PolygonFromText function creates a Polygon from a flat string of comma-separated coordinates. For a valid polygon, the sequence must form a closed ring by ensuring the last coordinate pair is identical to the first (e.g., `'x1, y1, x2, y2, x3, y3, x1, y1'`).

---

### Creating from Standard Formats

These functions parse common text-based or binary geospatial formats.

* `ST_GeomFromWKT(text)`: The primary function for constructing any geometry type from its **W**ell-**K**nown **T**ext (WKT) representation. This is one of the most common ways to ingest geometries.

* `ST_GeomFromWKB(binary)`: Creates a geometry from its **W**ell-**K**nown **B**inary (WKB) representation, which is a compact, machine-readable alternative to WKT.



---

### Creating from Other Geometries

This function combines existing geometries into a single feature.

* `ST_Collect(geometry_array)`: Takes an array of geometries and aggregates them into a single multi-part geometry (e.g., `MultiPoint`, `MultiPolygon`) or a `GeometryCollection`. This is useful for grouping related features together. This is an essential function because direct spatial operations on arrays are often limited, so ST_Collect consolidates the individual geometries into one object that can then be analyzed.

---

While these are common examples, they are not the only constructor functions available in WherobotsDB. We will now look at examples for `ST_MakePoint`, `ST_LineStringFromText`, `ST_Collect`, and `ST_MakeEnvelope`.

### ST_MakePoint()

In [ ]:
points_df = sedona.sql("""

SELECT ST_MakePoint(-122.349277, 47.620504) as space_needle, ST_MakePoint(-122.350446, 47.620556) as glass_museum, ST_MakePoint(-122.348258, 47.621494) as pop_culture_museum

""")

points_df.show(1, False)

In [ ]:
map_config_url = "https://raw.githubusercontent.com/wherobots/geospatial-data-engineering-associate/refs/heads/main/assets/week-1/conf/map_config.json"

with open(map_config_url, 'r') as file:
    map_config = json.load(file)

map = SedonaKepler.create_map(points_df, "Tourist spots", map_config)
map

### ST_LineStringFromText()

In [ ]:
line_df = sedona.sql("""

SELECT ST_LineStringFromText('-122.349277, 47.620504, -122.350446, 47.620556, -122.348258, 47.621494', ',') as order_to_visit

""")
line_df.show(1, False)

### ST_Collect()

In [ ]:
# PS. this allows us to access the dataframes in a SQL environment
points_df.createOrReplaceTempView("points")
line_df.createOrReplaceTempView("line")

collection_df = sedona.sql("""

SELECT 
    ST_Collect(Array(space_needle, glass_museum, pop_culture_museum, order_to_visit))
FROM points, line

""")

collection_df.show(1, False)

In [ ]:
map_collection = SedonaKepler.create_map(collection_df, "Things to do", map_config)
map_collection

### ST_MakeEnvelope()

In [ ]:
envelope_df = sedona.sql("""

SELECT ST_MakeEnvelope(-122.352848,47.619674,-122.346539,47.622451) AS tourist_location_bbox

""")

envelope_df.show(1, False)

# Wherobots Fundamentals - Spatial Predicates

Spatial predicates are functions that test the relationship between two geometries, returning `TRUE` or `FALSE`. They form the core of most spatial analysis, allowing you to filter data or create joins based on how geometries interact with each other. Understanding the exact logic of each predicate is key to performing accurate analysis.

In this section, we will explore some of the most essential predicate functions in detail.

-----

## ST_Intersects(A, B)

This is the most general-purpose spatial relationship, returning `TRUE` if two geometries **share any space at all**. This includes touching at a single point on their boundaries or overlapping in any way. It's the opposite of `ST_Disjoint`.

  * **Use Case:** Finding any parcels that intersect with a specific road.

-----

## ST_Contains(A, B) and ST_Within(A, B)

These two functions are opposites and describe a "spatially-inside" relationship.

  * `ST_Contains(A, B)` returns `TRUE` if geometry **A** completely encloses geometry **B**. No part of B can be outside of A. Think of a cookie inside a cookie jar; the jar contains the cookie.
  * `ST_Within(A, B)` returns `TRUE` if geometry **A** is completely inside geometry **B**. It's the reverse of `ST_Contains`. The cookie is within the jar.

A key detail is that the boundaries of the geometries cannot simply touch; at least one point of the inner geometry's interior must fall inside the outer geometry's interior. For example, a line that lies perfectly on the boundary of a polygon is not *contained* by it.

  * **Use Case:** Finding all the schools (`ST_Within`) a specific city district (`ST_Contains`).

-----

## ST_Overlaps(A, B)

This predicate is more specific than `ST_Intersects`. It returns `TRUE` only if two geometries **partially intersect** and are of the **same dimension**. For example, two overlapping polygons will return `TRUE`, but a line crossing a polygon will not. Critically, neither geometry can be completely contained within the other.

  * **Use Case:** Finding sales regions that have a partial overlap, which might indicate a territory dispute.

-----

### ST_DWithin(A, B, distance, [useSpheroid])

Instead of just testing a direct spatial relationship, `ST_DWithin` checks for **proximity**. It returns `TRUE` if the two geometries are **within a specified distance** of each other. This is extremely powerful for "buffer" style queries and is highly optimized to use spatial indexes, making it much faster than calculating the exact distance for every pair of geometries.

#### Distance Calculation: Spheroid vs. Euclidean

The optional `useSpheroid` flag is crucial as it controls how the distance is calculated:

* **`useSpheroid = true` (Spheroidal Distance):** This method should be used for geographic data (latitude/longitude). It calculates the more accurate "great-circle" distance on a curved surface. When this is enabled, the distance unit is always in **meters**, and the calculation is performed between the centroids of the two geometries.

* **`useSpheroid = false` (Euclidean Distance):** This is the default behavior. It performs a simpler, "flat-earth" distance calculation. The unit of the `distance` parameter in this case is the same as the unit of the data's Coordinate Reference System (CRS). For accurate results, you should first transform your data into a projected CRS appropriate for distance measurements (e.g., a UTM or State Plane system).

* **Use Case:** Finding all ATMs within 500 **meters** of a specific address using geographic coordinates (`useSpheroid = true`), or finding all competing stores within 2,500 **feet** of a location using a projected State Plane CRS where the units are in feet (`useSpheroid = false`).

-----

These are just a few of the many powerful predicate functions available in WherobotsDB. You can find the complete list in the [official documentation](https://docs.wherobots.com/latest/references/wherobotsdb/vector-data/Predicate/).

# Wherobots Fundamentals - Spatial Joins (Range Joins)

Now that you understand spatial predicates, you can use them to perform one of the most powerful operations in geospatial analysis: the **spatial join**. While a standard join uses a key like an ID to match rows (`tableA.id = tableB.id`), a spatial join combines data from two tables based on the spatial relationship between their geometries. This is a type of "range join" where the condition isn't simple equality but a spatial test, such as `ST_Intersects(A.geom, B.geom)`.

The primary goal of a spatial join is enrichment: adding attributes from one spatial dataset to another based on their shared location.

---

## Why Spatial Joins Matter

Spatial joins allow you to get insights from your data **as if location itself were a column you could join on**. They let you combine completely different datasets using their shared space as the common link, which unlocks powerful analytical capabilities.

### Contextual Enrichment

Imagine you have a table of customer addresses (points) and a separate table of county demographics (polygons). These tables have no common ID column. A spatial join lets you "enrich" your customer data by transferring the demographic information from the county polygon that each customer point falls within. You could then analyze customer behavior by county income level, population density, or any other demographic metric.



### Answering Complex Questions

Ultimately, spatial joins are how you answer real-world questions that involve location. For example:
* Which of our stores are located in flood-prone areas?
* What is the average property value for parcels within 500 meters of a new transit line?
* How many competitors are within a 10-minute drive of each of our locations?

In [ ]:
places_df = (
    sedona.sql("""

    WITH seattle_downtown AS (
        SELECT ST_GeomFromWKT('POLYGON ((-122.360916 47.590189, -122.299461 47.590189, -122.299461 47.641104, -122.360916 47.641104, -122.360916 47.590189))') AS geom
    ),

    -- This is to leverage spatial predicate pushdown
    places AS (
      SELECT *
      FROM
        wherobots_open_data.overture_maps_foundation.places_place places, seattle_downtown
      WHERE
        ST_Intersects(places.geometry, seattle_downtown.geom)
    ),

    -- This is to leverage spatial predicate pushdown
    buildings AS (
      SELECT *
      FROM
        wherobots_open_data.overture_maps_foundation.buildings_building buildings, seattle_downtown
      WHERE
        ST_Intersects(buildings.geometry, seattle_downtown.geom)
    )
    
    SELECT
      places.names.primary as place_name,
      places.categories.primary as place_type,
      element_at(places.addresses, 1) as place_address,
      ROUND(places.confidence * 100, 2) AS `place_confidence (%)`,
      places.geometry as place_geometry,
      buildings.geometry as building_geometry
    FROM
      places
    JOIN
      buildings
    ON
      ST_Intersects(places.geometry, buildings.geometry);
    
    """)
    # We are caching the result, as we will reuse it to visualize the data
        .cache()
)

places_df.show(10, False)

In [ ]:
map_config_join_url = "https://raw.githubusercontent.com/wherobots/geospatial-data-engineering-associate/refs/heads/main/assets/week-1/conf/map_config_join.json"

with open(map_config_url, 'r') as file:
    map_config = json.load(file)

map_places = SedonaKepler.create_map(points_df, "Places in buildings", map_config)
map_places

In [ ]:
# create and manage Havasu (Iceberg) tables for vector and raster data  - Furqaan

# Creating Havasu Tables from Sedona DataFrames

Now that you understand the importance of Apache Iceberg, let's see how simple it is to save a Sedona DataFrame as an Iceberg table. Wherobots uses an enhanced version of Iceberg called **Havasu**, which is purpose-built for high-performance geospatial analytics.

---

## What is Havasu?

Standard Apache Iceberg did not natively support geometry data types until its v3 specification. **Havasu** is Wherobots' enhanced implementation of Iceberg that adds first-class support for both **vector** and **raster** data. This allows you to combine all the benefits of the Iceberg format—like atomic transactions and schema evolution—with native, high-performance geospatial data handling.

---

## Saving a DataFrame to Havasu

Saving a Sedona DataFrame as a Havasu table is a straightforward, one-line command. The process is identical whether your DataFrame contains vector geometries, rasters, or no spatial data at all.

In [ ]:
# Create a new Havasu (Iceberg) database

database = 'gde_bronze'

sedona.sql(f'CREATE DATABASE IF NOT EXISTS wherobots.{database}')

In [ ]:
geotiff_path = "s3://wherobots-examples/data/ghs_population/GHS_POP_E1975_GLOBE_R2023A_4326_3ss_V1_0.tif"

df = sedona.read.format("raster") \
    .option("tileWidth", "512") \
    .option("tileHeight", "512") \
    .option("retile", "true") \
    .load(geotiff_path)

df.writeTo(f"wherobots.{database}.ghs_population_tiles")

# Data validity checks

Two of the most common issues with geospatial data include managing projections or Coordinate Reference Systems (CRS) and ensuring geometries are valid.

- A geometry is invalid if it violates spatial rules like self-intersections, unclosed rings, misaligned holes, or overlapping parts—making it topologically incorrect.
- Spatial files generally contain a Coordinate Reference System or CRS that is defined by a Spatial Reference ID or SRID. This tells us how the data is projected from the round spheroid of the earth onto a flat surface.

To fix these issues and ensure our data is valid and in the correct format we use two approaches:

1. Check the geometries for any invalidities, and if there are attempt to fix them using `ST_IsValid`, `ST_IsValidDetail`, and `ST_MakeValid`
2. Remove or log out any geometries that cannot be fixed
3. Standardize our geometries in a single CRS, in this case [EPSG:4326](https://epsg.io/4326) which renders in a coordinate reference system

## Validating geometries

In [ ]:
# Data validity checks - Matt

## Transforming CRS

In [ ]:
# Handling and transforming CRS - Matt

In [ ]:
# Dataset loading - aka load all datasets to tables - Matt

# Loading datasets into WherobotsDB

In [ ]:
prefix = 's3://wherobots-examples/gdea-course-data/raw-data/'

In [6]:
def check_invalid_geometries(df: DataFrame, geom_col: str = "geom", reason_col: str = "why_invalid") -> int:
    df_with_reason = df.withColumn(reason_col, ST_IsValidReason(col(geom_col)))
    # cache to avoid recomputation if you inspect reasons later
    df_with_reason.cache()
    invalid_count = df_with_reason.filter(~ST_IsValid(col(geom_col))).count()
    print(f"✅ Checked geometries — found {invalid_count} invalid geometries.")
    return invalid_count

def fix_invalid_geometries(df: DataFrame, invalid_count: int, geom_col: str = "geom") -> DataFrame:
    if invalid_count > 1:
        print(f"🔧 Attempting to fix {invalid_count} invalid geometries...")
        return df.withColumn(
            geom_col,
            when(~ST_IsValid(col(geom_col)), ST_MakeValid(col(geom_col))).otherwise(col(geom_col))
        )
    else:
        print("⚡ Only one invalid geometry (or none). Skipping automated fix.")
        return df

# --- driver program ---
def process_geometries(
    df: DataFrame,
    geom_col: str = "geom",
    attempt_fix: bool = True,
    split_on_fail: bool = True
):
    """
    Runs validity check -> optional repair -> optional split.
    Returns either:
      - {"df": corrected_df}  when all geometries valid after repair (or none invalid)
      - {"valid_df": ..., "invalid_df": ...} when some invalid remain and split_on_fail=True
    """
    # 1) Initial check
    invalid_count = check_invalid_geometries(df, geom_col=geom_col)

    if invalid_count == 0:
        print("✅ All geometries are valid.")
        return {"df": df}  # nothing to do

    # 2) Attempt repair (only changes rows that are invalid per your earlier contract)
    if attempt_fix:
        df_fixed = fix_invalid_geometries(df, invalid_count, geom_col=geom_col)
        remaining_invalid_count = df_fixed.filter(~ST_IsValid(col(geom_col))).count()
        print(f"🔎 After fixing, {remaining_invalid_count} invalid geometries remain.")
    
        if remaining_invalid_count == 0:
            print("✅ All geometries are valid after fixing.")
            return {"df": df_fixed}
        elif split_on_fail:
            print("⚠️ Some invalid geometries remain — splitting dataset.")
            valid_df = df_fixed.filter(ST_IsValid(col(geom_col)))
            invalid_df = df_fixed.filter(~ST_IsValid(col(geom_col)))
            print(f"✅ Split complete: {valid_df.count()} valid / {invalid_df.count()} invalid.")
            return {"valid_df": valid_df, "invalid_df": invalid_df}
        else:
            print("⚠️ Some invalid geometries remain, returning best-effort fixed DataFrame.")
            return {"df": df_fixed}
    
    # If no fix attempt, just split if requested
    if split_on_fail:
        print("⚠️ Skipping fix — splitting into valid and invalid.")
        valid_df = df.filter(ST_IsValid(col(geom_col)))
        invalid_df = df.filter(~ST_IsValid(col(geom_col)))
        print(f"✅ Split complete: {valid_df.count()} valid / {invalid_df.count()} invalid.")
        return {"valid_df": valid_df, "invalid_df": invalid_df}
    
    print("⚠️ Invalid geometries found but no fix or split requested. Returning original DataFrame.")
    return {"df": df}

In [7]:
# FEMA Flood Hazard Areas
fld_hazard_area = sedona.read.format('shapefile').load(f'{prefix}' + '53033C_20250330/S_FLD_HAZ_AR.shp')

In [40]:
result = process_geometries(fld_hazard_area, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

25/09/30 18:43:28 WARN CacheManager: Asked to cache already cached data.


✅ Checked geometries — found 15 invalid geometries.
🔧 Attempting to fix 15 invalid geometries...


🔎 After fixing, 0 invalid geometries remain.
✅ All geometries are valid after fixing.


In [41]:
df_final.writeTo(f"wherobots.{database}.fema_flood_zones_bronze").createOrReplace()

In [42]:
# King County Generalized Land Use Data
gen_land_use = sedona.read.format('shapefile').load(f'{prefix}' + 'General_Land_Use_Final_Dataset/General_Land_Use_Final_Dataset.shp')

In [43]:
result = process_geometries(gen_land_use, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 2987 invalid geometries.
🔧 Attempting to fix 2987 invalid geometries...


🔎 After fixing, 0 invalid geometries remain.
✅ All geometries are valid after fixing.


In [44]:
valid_df.writeTo(f"wherobots.{database}.gen_land_use_bronze").createOrReplace()

In [45]:
# King County Sherrif Patrol Districts
sherrif_districts = sedona.read.format('shapefile').load(f'{prefix}' + 'King_County_Sheriff_Patrol_Districts___patrol_districts_area/King_County_Sheriff_Patrol_Districts___patrol_districts_area.shp')

In [46]:
result = process_geometries(sherrif_districts, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 0 invalid geometries.
✅ All geometries are valid.


In [47]:
df_final.writeTo(f"wherobots.{database}.sherrif_districts_bronze").createOrReplace()

In [48]:
offense_reports = sedona.read.format('csv').load(f'{prefix}' + 'KCSO_Offense_Reports__2020_to_Present_20250923.csv')

In [49]:
offense_reports.writeTo(f"wherobots.{database}.offense_reports_bronze").createOrReplace()

In [50]:
# King County Bike Lanes
bike_lanes = sedona.read.format('shapefile').load(f'{prefix}' + 'Metro_Transportation_Network_(TNET)_in_King_County_for_Bicycle_Mode___trans_network_bike_line/Metro_Transportation_Network_(TNET)_in_King_County_for_Bicycle_Mode___trans_network_bike_line.shp')

In [51]:
result = process_geometries(bike_lanes, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 0 invalid geometries.
✅ All geometries are valid.


In [52]:
df_final.writeTo(f"wherobots.{database}.bike_lanes_bronze").createOrReplace()

In [53]:
# FEMA National Risk Index
fema_nri = sedona.read.format('shapefile').load(f'{prefix}' + 'NRI_Shapefile_CensusTracts/NRI_Shapefile_CensusTracts.shp')

In [56]:
result = process_geometries(fema_nri, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

25/09/30 19:01:47 WARN CacheManager: Asked to cache already cached data.


✅ Checked geometries — found 82 invalid geometries.
🔧 Attempting to fix 82 invalid geometries...


🔎 After fixing, 0 invalid geometries remain.
✅ All geometries are valid after fixing.


In [57]:
df_final.writeTo(f"wherobots.{database}.fema_nri_bronze").createOrReplace()

25/09/30 19:19:03 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed.


In [8]:
# King County School Sites
school_sites = sedona.read.format('shapefile').load(f'{prefix}' + 'School_Sites_in_King_County___schsite_point/School_Sites_in_King_County___schsite_point.shp')

In [9]:
result = process_geometries(school_sites, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 0 invalid geometries.
✅ All geometries are valid.


In [10]:
df_final.writeTo(f"wherobots.{database}.school_sites_bronze").createOrReplace()

In [20]:
# Schools Report Card
report_card = sedona.read. \
    format('csv'). \
    load(f'{prefix}' + 'Report_Card_Growth_for_2024-25_20250923.csv')

In [21]:
report_card.writeTo(f"wherobots.{database}.report_card_bronze").createOrReplace()

In [22]:
# Seismic Hazards
seismic_hazards = sedona.read. \
    format('shapefile'). \
    load(f'{prefix}' + 'Seismic_Hazards___seism_area/Seismic_Hazards___seism_area.shp')

In [23]:
result = process_geometries(seismic_hazards, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 0 invalid geometries.
✅ All geometries are valid.


25/10/01 02:01:27 WARN CacheManager: Asked to cache already cached data.


In [24]:
df_final.writeTo(f"wherobots.{database}.seismic_hazards_bronze").createOrReplace()

In [25]:
# Census Block Groups
block_groups = sedona.read. \
    format('shapefile'). \
    load(f'{prefix}' + 'tl_2024_53_bg/tl_2024_53_bg.shp')

In [26]:
result = process_geometries(block_groups, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

25/10/01 02:01:54 WARN CacheManager: Asked to cache already cached data.


✅ Checked geometries — found 0 invalid geometries.
✅ All geometries are valid.


In [27]:
df_final.writeTo(f"wherobots.{database}.block_groups_bronze").createOrReplace()

In [28]:
# Census CSVs
median_age = sedona.read. \
    format('csv'). \
    load(f'{prefix}' + 'ACSDT5Y2023.B01002_2025-09-19T105233/ACSDT5Y2023.B01002-Data.csv')

median_age.writeTo(f"wherobots.{database}.median_age_bronze").createOrReplace()

total_pop = sedona.read. \
    format('csv'). \
    load(f'{prefix}' + 'ACSDT5Y2023.B01003_2025-09-19T105050/ACSDT5Y2023.B01003-Data.csv')

total_pop.writeTo(f"wherobots.{database}.total_pop_bronze").createOrReplace()

median_income = sedona.read. \
    format('csv'). \
    load(f'{prefix}' + 'ACSDT5Y2023.B19013_2025-09-19T105253/ACSDT5Y2023.B19013-Data.csv')

total_pop.writeTo(f"wherobots.{database}.median_income_bronze").createOrReplace()

In [29]:
# Tranist Routes
transit_routes = sedona.read. \
    format('shapefile'). \
    load(f'{prefix}' + 'Transit_Routes_for_King_County_Metro___transitroute_line/Transit_Routes_for_King_County_Metro___transitroute_line.shp')

In [30]:
result = process_geometries(transit_routes, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 0 invalid geometries.
✅ All geometries are valid.


In [31]:
df_final.writeTo(f"wherobots.{database}.transit_routes_bronze").createOrReplace()

In [32]:
# Transit Stops
transit_stops = sedona.read. \
    format('shapefile'). \
    load(f'{prefix}' + 'Transit_Stops_for_King_County_Metro___transitstop_point/Transit_Stops_for_King_County_Metro___transitstop_point.shp')

In [33]:
result = process_geometries(transit_stops, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 0 invalid geometries.
✅ All geometries are valid.


In [34]:
df_final.writeTo(f"wherobots.{database}.transit_stops_bronze").createOrReplace()

In [35]:
# Water Bodies
water_bodies = sedona.read. \
    format('shapefile'). \
    load(f'{prefix}' + 'Waterbodies_with_History_and_Jurisdictional_detail___wtrbdy_det_area/Waterbodies_with_History_and_Jurisdictional_detail___wtrbdy_det_area.shp')

In [36]:
result = process_geometries(water_bodies, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 5 invalid geometries.
🔧 Attempting to fix 5 invalid geometries...


🔎 After fixing, 0 invalid geometries remain.
✅ All geometries are valid after fixing.


In [37]:
df_final.writeTo(f"wherobots.{database}.water_bodies_bronze").createOrReplace()

In [39]:
# Wildfire Polygons
wildfires = sedona.read. \
    format('shapefile'). \
    load(f'{prefix}' + 'Wildfires_1878_2019_Polygon_Data/Shapefile/US_Wildfires_1878_2019.shp')

In [40]:
result = process_geometries(wildfires, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

✅ Checked geometries — found 1203 invalid geometries.
🔧 Attempting to fix 1203 invalid geometries...


🔎 After fixing, 0 invalid geometries remain.
✅ All geometries are valid after fixing.


In [41]:
df_final.writeTo(f"wherobots.{database}.wildfires_bronze").createOrReplace()

In [11]:
# Wildfire Rasters

In [12]:
# Elevation

# https://s3.opengeohub.org/global/edtm/gedtm_rf_m_30m_s_20060101_20151231_go_epsg.4326.3855_v20250611.tif	

In [50]:
# Geocoded Schools
schools = sedona.read. \
    format('geojson'). \
    load(f'{prefix}' + 'Washington_State_Public_Schools_GeoCoded.geojson')

In [54]:
schools = schools \
    .withColumn("geometry", expr("geometry")) \
    .withColumn("AYPCode", expr("properties['AYPCode']")) \
    .withColumn("CongressionalDistrict", expr("properties['CongressionalDistrict']")) \
    .withColumn("County", expr("properties['County']")) \
    .withColumn("ESDCode", expr("properties['ESDCode']")) \
    .withColumn("ESDName", expr("properties['ESDName']")) \
    .withColumn("Email", expr("properties['Email']")) \
    .withColumn("GeoCoded_X", expr("properties['GeoCoded_X']")) \
    .withColumn("GeoCoded_Y", expr("properties['GeoCoded_Y']")) \
    .withColumn("GradeCategory", expr("properties['GradeCategory']")) \
    .withColumn("HighestGrade", expr("properties['HighestGrade']")) \
    .withColumn("LEACode", expr("properties['LEACode']")) \
    .withColumn("LEAName", expr("properties['LEAName']")) \
    .withColumn("LegislativeDistrict", expr("properties['LegislativeDistrict']")) \
    .withColumn("LowestGrade", expr("properties['LowestGrade']")) \
    .withColumn("MailingAddress", expr("properties['MailingAddress']")) \
    .withColumn("NCES_X", expr("properties['NCES_X']")) \
    .withColumn("NCES_Y", expr("properties['NCES_Y']")) \
    .withColumn("Phone", expr("properties['Phone']")) \
    .withColumn("Principal", expr("properties['Principal']")) \
    .withColumn("School", expr("properties['School']")) \
    .withColumn("SchoolCategory", expr("properties['SchoolCategory']")) \
    .withColumn("SchoolCode", expr("properties['SchoolCode']")) \
    .withColumn("SingleAddress", expr("properties['SingleAddress']")) \
    .drop("properties").drop("type") \
    .drop("_corrupt_record").drop("type") \
    .drop("type").drop("type")

In [58]:
schools.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- AYPCode: string (nullable = true)
 |-- CongressionalDistrict: string (nullable = true)
 |-- County: string (nullable = true)
 |-- ESDCode: long (nullable = true)
 |-- ESDName: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- GeoCoded_X: double (nullable = true)
 |-- GeoCoded_Y: double (nullable = true)
 |-- GradeCategory: string (nullable = true)
 |-- HighestGrade: string (nullable = true)
 |-- LEACode: long (nullable = true)
 |-- LEAName: string (nullable = true)
 |-- LegislativeDistrict: string (nullable = true)
 |-- LowestGrade: string (nullable = true)
 |-- MailingAddress: string (nullable = true)
 |-- NCES_X: double (nullable = true)
 |-- NCES_Y: double (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Principal: string (nullable = true)
 |-- School: string (nullable = true)
 |-- SchoolCategory: string (nullable = true)
 |-- SchoolCode: long (nullable = true)
 |-- SingleAddress: string (nullable = true

In [55]:
result = process_geometries(schools, geom_col="geometry", attempt_fix=True, split_on_fail=True)

if "df" in result:
    df_final = result["df"]  # all valid (either already valid or successfully repaired)
else:
    valid_df = result["valid_df"]
    invalid_df = result["invalid_df"]
    # handle invalids (e.g., export for manual review)(fld_hazard_area, 'geometry')

25/10/01 02:29:06 ERROR TaskSetManager: Task 0 in stage 84.0 failed 4 times; aborting job


Py4JJavaError: An error occurred while calling o494.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 84.0 failed 4 times, most recent failure: Lost task 0.3 in stage 84.0 (TID 1698) (10.1.31.162 executor 8): java.lang.NullPointerException: Cannot invoke "org.apache.spark.unsafe.types.UTF8String.toString()" because the return value of "org.apache.spark.sql.catalyst.InternalRow.getUTF8String(int)" is null
	at org.apache.spark.sql.catalyst.InternalRow.getString(InternalRow.scala:35)
	at org.apache.spark.sql.sedona_sql.io.geojson.GeoJSONUtils$.$anonfun$convertGeoJsonToGeometry$1(GeoJSONUtils.scala:111)
	at org.apache.spark.sql.sedona_sql.io.geojson.GeoJSONUtils$.$anonfun$convertGeoJsonToGeometry$1$adapted(GeoJSONUtils.scala:109)
	at scala.collection.ArrayOps$.foreach$extension(ArrayOps.scala:1328)
	at org.apache.spark.sql.sedona_sql.io.geojson.GeoJSONUtils$.convertGeoJsonToGeometry(GeoJSONUtils.scala:109)
	at org.apache.spark.sql.sedona_sql.io.geo

In [ ]:
df_final.writeTo(f"wherobots.{database}.schools_bronze").createOrReplace()